# Using Keras modules with TensorFlow

This notebook shows how to use Keras models with SageMaker using the TensorFlow container.
The model used for this notebook is a simple deep CNN model, trained with the CIFAR-10 dataset. The example was extracted from [Keras examples](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py).

### Set up the environment

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

### Download the CIFAR-10 dataset
Downloading the test and training data will take around 5 minutes.

In [ ]:
import utils

utils.cifar10_download()

### Upload the data to a S3 bucket

In [ ]:
inputs = sagemaker_session.upload_data(path='/tmp/cifar10_data', key_prefix='data/DEMO-cifar10')

**sagemaker_session.upload_data** will upload the CIFAR-10 dataset from your machine to a bucket named **sagemaker-{region}-{*your aws account number*}**, if you don't have this bucket yet, sagemaker_session will create it for you.

### Complete source code
- [cifar10_cnn.py](cifar10_cnn.py): Keras model

In [ ]:
!cat cifar10_cnn.py

## Create a training job using the sagemaker.TensorFlow estimator

In [ ]:
from sagemaker.tensorflow import TensorFlow


estimator = TensorFlow(entry_point='cifar10_cnn.py',
                       role=role,
                       hyperparameters={'min_eval_frequency': 10},
                       training_steps=1000, evaluation_steps=100,
                       train_instance_count=1, train_instance_type='ml.c4.xlarge')

estimator.fit(inputs)

**Atention**: Keras modules have a known issue and cannot be issue for distributed (multi-instances) training.
Keep **train_instance_count** == 1 until they the TensorFlow/Keras team, support this feature.
More information about the issue: https://github.com/tensorflow/tensorflow/issues/14504


# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint which serves prediction requests in real-time.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

## Making some predictions

In [ ]:
# Creating fake prediction data
import numpy as np
data = np.random.randn(10, 32, 32, 3)
predictor.predict(data)

# Cleaning up
To avoid incurring charges to your AWS account for the resources used in this tutorial you need to delete the **SageMaker Endpoint:**

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)